In [1]:
import os
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.append(p)

from waveorder.io.writer import WaveorderWriter
import numpy as np
import zarr

### Initialize the writer

This will instantiate the writer class, no zarr stores, subfolders, etc have been created yet.  This step allows you to define which type of writer you want and let it know your directory to save the data

In [2]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop/zarr_test', 'physical')
writer_stokes = WaveorderWriter('/Users/cameron.foltz/Desktop/zarr_test', 'stokes')

### Set Position and Create Zarr

'set_position()' will create a subfolder in your save directory with the corresponding position number.  The current directory of the writer is now this subfolder.

'create_zarr()' will create an empty zarr store within the position subfolder.  No arrays have been added here.  Name of the store can be input by the user.  If no name is provided, store will be named 'physical_data.zarr' or 'stokes_data.zarr'

The current writer.store will now be this newly created zarr store

In [5]:
writer.set_position(0)
writer.create_zarr(name=None)

writer.set_position(1)
writer.create_zarr(name=None)

writer_stokes.set_position(0)
writer_stokes.create_zarr(name=None)

### Change to an existing zarr store

Useful function for when you are writing to different stores at different times.  i.e. 'position first' writing (example later)

In [21]:
print(f'Current store is at: {writer.position_dir}/physical_data.zarr')

writer.set_position(0)
writer.set_zarr(os.path.join(writer.position_dir,'physical_data.zarr'))

Current store is at: /Users/cameron.foltz/Desktop/zarr_test/Pos_001/physical_data.zarr
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_000/physical_data.zarr


### Create some fake data to save later

In [25]:
P = 10
T = 3
C = 2
Z = 65
Y = 256
X = 256

data = np.random.rand(P, T, C, Z, Y, X)

In [26]:
np.shape(data)

(10, 3, 2, 65, 256, 256)

### initialize the array which sets the data saving parameters

Here is where you will initialize the array in your current writer.store .  data_shape should match the (T, C, Z, Y, X) dimensions of your data.  Positions need to be separated by subfolder.

chunk_size determines how zarr will chunk your data.  This means that when you later try to load the data, it will load one chunk at a time with this specified size.  To have the chunk be one z-slice, you would set chunk_size = (1,1,1,Y,X)

In [30]:
writer.init_array(data_shape=(3,2,65,256,256), chunk_size=(1,1,1,256,256), dtype='float32')
writer_stokes.init_array(data_shape=(3,4,65,256,256), chunk_size=(1,1,1,256,256), dtype='float32')

### set the channel zarr_attributes

This will set the ome-zarr metadata for each of your channels.  Name list must match the channels you are saving. i.e. if your first channel (C=0) is Phase3D, start the list with 'Phase3D'.  There are pre-populated metadata settings for specific names (i.e. 'Retardance', 'Orientation', 'S0').  If name is not reconginzed, default settings are used.

In [31]:
writer.set_channel_attributes(['DAPI', 'Phase3D'])
writer_stokes.set_channel_attributes(['S0', 'S1', 'S2', 'S3'])

In [32]:
# print zarr_attributes dictionary
writer.store.attrs.asdict()

{'multiscales': [{'datasets': [{'path': 'array'}], 'version': '0.1'}],
 'omero': {'channels': [{'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'DAPI',
    'window': {'end': 10000.0, 'max': 10000.0, 'min': 0.0, 'start': 0.0}},
   {'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'Phase3D',
    'window': {'end': 0.2, 'max': 1.0, 'min': -1.0, 'start': -0.2}}],
  'rdefs': {'defaultT': 0,
   'model': 'color',
   'projection': 'normal',
   'defaultZ': 0},
  'version': 0.1}}

In [33]:
writer_stokes.store.attrs.asdict()

{'multiscales': [{'datasets': [{'path': 'array'}], 'version': '0.1'}],
 'omero': {'channels': [{'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'S0',
    'window': {'end': 1.0, 'max': 10, 'min': 0.0, 'start': 0.0}},
   {'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'S1',
    'window': {'end': 0.5, 'max': 1.0, 'min': -1.0, 'start': -0.5}},
   {'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'S2',
    'window': {'end': 0.5, 'max': 1.0, 'min': -1.0, 'start': -0.5}},
   {'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'S3',
    'window': {'end': 1.0, 'max': 1.0, 'min': -1.0, 'start': -1.0}}],
  'rdefs': {'defaultT': 0,
   'model': 'color',
   'projection': 'normal',
   'defaultZ': 0},
  'versi

### Write the data along specific dimensions

In [11]:
writer.write(data, t=[0,3], c=[0,2], z=[0,65])

# Complex, 'Position First' Acquisition

Let's say you have a dataset with 10 position with each position containing a data of size (T, C, Z, Y, X) = (3, 2, 65, 256, 256) and you want to save the first time point of every position first, and then move on to the next timepoint.  This will show you how to properly loop through this example case.

In [43]:
# Must first initialize all of the arrays, since you will be returning to them 
# at different times in the data-saving process.
num_pos = 10

for i in range(num_pos):
    writer.set_position(i)
    writer.create_zarr()
    writer.init_array(data_shape=(3,2,65,256,256), chunk_size=(1,1,1,256,256), dtype='float32')
    writer.set_channel_attributes(['Retardance', 'Orientation'])
    
    
# Now begin the rime loop 
for t in range(3):
    
    # At each time point, write data for all positions
    for pos in range(num_pos):
        
        print(f'Writing Position {pos} at Time {t}')
        
        #change to the correct position folder
        writer.set_position(pos)
        
        #change to the zarr store in this position folder
        writer.set_zarr(os.path.join(writer.position_dir,'physical_data.zarr'))

        #write this position data
        writer.write(data[pos, t], t=t, c=[0,2], z=[0,65])

Writing Position 0 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_000/physical_data.zarr
Writing Position 1 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_001/physical_data.zarr
Writing Position 2 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_002/physical_data.zarr
Writing Position 3 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_003/physical_data.zarr
Writing Position 4 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_004/physical_data.zarr
Writing Position 5 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_005/physical_data.zarr
Writing Position 6 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_006/physical_data.zarr
Writing Position 7 at Time 0
Opening existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_007/physical_data.zarr
Writing Position 8 at Time 0
Ope